In [90]:
using JuMP
using Gurobi

global arcs = [1 2; 1 3; 1 4; 2 3; 2 5; 3 5; 4 5]
# global paths = [[1 2 3 5],
#                 [1 2 5],
#                 [1 3 5],
#                 [1 4 5]] 

global N = collect(1:maximum(arcs))
origin = 1
destination = 5
global arcNum = length(arcs[:,1])

#The SP#
function LPMin(x, origin, destination,p)
    global arcs
    global N
    global arcNum

    gurobi_env = Gurobi.Env()
    setparams!(gurobi_env, Heuristics=0.0, Cuts = 0, OutputFlag = 0)
    m = Model(() -> Gurobi.Optimizer(gurobi_env))
    
    #Flow variables
    @variable(m, y[1:arcNum]>=0)
    #Max capacity variable
    @variable(m, u)
    
    flow_cons = Array{JuMP.ConstraintRef}(undef, length(N)-1)
    ucap_cons = Array{JuMP.ConstraintRef}(undef, arcNum)
    xcap_cons = Array{JuMP.ConstraintRef}(undef, arcNum)

    #Flow constraints
    for i in N
        #Outgoing flows from source node add to 1
        if i == origin
            outgoing = findall(arcs[:,1].== origin)
            flow_cons[i] = @constraint(m, sum(y[j] for j in outgoing) == 1)
        end
        #Outflow - Inflow = 0 for all nodes that are neither source nor sink
        if i != origin && i != destination  
            outgoing = findall(arcs[:,1].== i)
            incoming = findall(arcs[:,2].== i)
            flow_cons[i] = @constraint(m, sum(y[j] for j in outgoing)-sum(y[j] for j in incoming) == 0)
        end
    end

    #Constraints y_k <= (1/p_k) * v
    #If p_k = 0, then y_k <= infty * v
    #y_k is then not dependent on v  
    #v can be as small as possible for any value of y_k in this case
    #So only constraints for non-zero p_k are added
    
    #At the moment, all p_k > 0
    #Size of ucap_cons needs to be adjusted if we allow some p_k to be 0
    for i = 1:arcNum
        if p[i] != 0
            ucap_cons[i] = @constraint(m, u*1/p[i] >= y[i])
        end
    end

    #Constraints y_k <= 1-x_k
    #x here is an input and not a variable
    for i = 1:arcNum 
        xcap_cons[i] = @constraint(m, 1-x[i]>= y[i])  
    end

    @objective(m, Min, u)
    optimize!(m)
    
    
    current_Obj = JuMP.objective_value.(m)
    current_y = JuMP.value.(y)
    current_u = JuMP.value.(u)
    flow_dual = zeros(length(N)-1)
    u_dual = zeros(arcNum)
    xcap_dual = zeros(arcNum)
    
    
    for i = 1:arcNum
        xcap_dual[i] = JuMP.dual(xcap_cons[i]) #Duals corresponding to y_k <= 1-x_k
        u_dual[i] = JuMP.dual(ucap_cons[i]) #Duals corresponding to y_k <= v/p_k
    end
    
    for i = 1:length(N)-1
        flow_dual[i] = JuMP.dual(flow_cons[i])
    end
    
    println("SP Obj = ",current_Obj)
    println("y_sol = ", current_y)
    println("flow_dual", flow_dual)
    println("u_dual", u_dual)
    println("xcap_dual", xcap_dual)

    return current_Obj, flow_dual, u_dual, xcap_dual
end

x_now = zeros(arcNum)#[0,0,0]
x_now[1] = 1
obj, flow_dual, u_dual, xcap_dual = LPMin(x_now, origin, destination)

Academic license - for non-commercial use only
SP Obj = 0.5
flow_dual[0.5, 0.0, 0.5, 0.5]
u_dual[0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5]
xcap_dual[0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


(0.5, [0.5, 0.0, 0.5, 0.5], [0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5], [0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [91]:
#Generate scenarios:
numScen = 3
p = Array{Float64}(undef, numScen, arcNum)
# for i = 1:numScen
#     rowTotal = 0
#     for j = 1:arcNum
#         r = rand(1:5)
#         p[i,j] = r
# #         rowTotal = rowTotal + r
#     end
#     p[i,:] = 1 ./ p[i,:] #./rowTotal
# end

#An instance of p:
p = [1.0 0.25 0.333333 0.25 0.2 1.0 0.5; 
    0.5 0.25 0.333333 0.5 0.5 1.0 0.5; 
    0.5 1.0 0.5 0.5 1.0 1.0 0.2]


# p = [1/64 1/32 1/4 1/2 1/8 1/16 1/64;
#     0 1/5 1/5 1/5 0 1/5 1/5;
#     1/4 0 1/5 0 1/5 1/10 1/4]

#Define MP:
global c = [5,14,15,6,12,15,7] #interdiction cost
global b = 10 #budget
gurobi_env = Gurobi.Env()
setparams!(gurobi_env, Heuristics=0.0, Cuts = 0, OutputFlag = 0)
MP = Model(() -> Gurobi.Optimizer(gurobi_env))

@variable(MP, x[1:arcNum], Bin)
@variable(MP, θ[1:numScen] >= -1e6)

#Budget Constraints
@constraint(MP, sum(c[i]*x[i] for i=1:arcNum) <= 8)

@objective(MP, Min, (1/numScen)*sum(θ[i] for i = 1:numScen))

last_Obj = -20
MP_Obj = -10
iter = 1

#At the moment: Make sure to have at least 2 iterations in to have 
#decent values for last_Obj and MP_Obj
while MP_Obj - last_Obj > 1 || iter <= 2
    
    last_Obj = MP_Obj
    optimize!(MP)
    println("\n\nIter ", iter)

    MP_Obj = JuMP.objective_value.(MP)
    x_now = JuMP.value.(x)
    println("Obj = ",MP_Obj)
    println("x = ", x_now,"\n\n")
    
    #Solve SP for each scenario
    for k = 1:numScen
        println("\nk = ", k)
        
        obj, flow_dual, u_dual, xcap_dual = LPMin(x_now, origin, destination, p[k,:])
        cons = @constraint(MP, θ[k] >= flow_dual[1]*1 + sum(xcap_dual[i]*(1-x[i]) for i = 1:arcNum)) 
        println(cons)
    end
    
    println("last_Obj = ", last_Obj)
    println("MP_Obj = ", MP_Obj)
    iter = iter+1 
end

7
-1
0
p = [1.0 0.25 0.333333 0.25 0.2 1.0 0.5; 0.5 0.25 0.333333 0.5 0.5 1.0 0.5; 0.5 1.0 0.5 0.5 1.0 1.0 0.2]
Academic license - for non-commercial use only


Iter 1
Obj = -1.0e6
x = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]



k = 1
[1.0, 0.25, 0.333333, 0.25, 0.2, 1.0, 0.5]
Academic license - for non-commercial use only
SP Obj = 0.25
y_sol = [0.25, 0.25, 0.5, 0.0, 0.25, 0.25, 0.5]
flow_dual[0.25, 0.0, 0.25, 0.25]
u_dual[0.25, 0.0, 0.0, 0.0, 0.0, 0.25, 0.25]
xcap_dual[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
θ[1] >= 0.25

k = 2
[0.5, 0.25, 0.333333, 0.5, 0.5, 1.0, 0.5]
Academic license - for non-commercial use only
SP Obj = 0.19999999999999998
y_sol = [0.4, 0.2, 0.4, 0.0, 0.4, 0.2, 0.4]
flow_dual[0.2, 0.0, 0.2, 0.2]
u_dual[0.2, 0.0, 0.0, 0.0, 0.0, 0.2, 0.2]
xcap_dual[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
θ[2] >= 0.19999999999999998

k = 3
[0.5, 1.0, 0.5, 0.5, 1.0, 1.0, 0.2]
Academic license - for non-commercial use only
SP Obj = 0.25
y_sol = [0.25, 0.25, 0.5, 0.0, 0.25, 0.25, 0.5]
flow_dual[0.25, 